In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
data = Path("../data")
plot_path = data / "plots"
map_path = data / "graph3_map"
stat_path = map_path / "stats"
geojson_path = map_path / "geojson"

import json

# Load geojson data


In [2]:
# Load custom.geo.json
with open(geojson_path / "custom.geo.json") as f:
    geojson = json.load(f)


# Each of the feature in "features" jas a "properties" fields, and inside of this field an "admin" fields. Put all these in a dataframe
df_geojson = pd.DataFrame(
    [
        {
            "country": feature["properties"]["admin"],
        }
        for feature in geojson["features"]
    ]
)
df_geojson["country"].sort_values()

89        Afghanistan
15            Albania
156           Algeria
152            Angola
41          Argentina
            ...      
132    Western Sahara
93              Yemen
139            Zambia
153          Zimbabwe
158          eSwatini
Name: country, Length: 179, dtype: object

In [3]:
with open(geojson_path / "countries.geo.json") as f:
    countries_geo_json = json.load(f)

df_countries_geo_json = pd.DataFrame(
    [
        {
            "country": feature["properties"]["name"],
        }
        for feature in countries_geo_json["features"]
    ]
)
df_countries_geo_json["country"].sort_values()

0         Afghanistan
2             Albania
45            Algeria
1              Angola
6          Antarctica
            ...      
172         West Bank
136    Western Sahara
173             Yemen
175            Zambia
176          Zimbabwe
Name: country, Length: 177, dtype: object

In [6]:
with open(geojson_path / "countries.geojson") as f:
    countries_geojson = json.load(f)

df_countries_geojson = pd.DataFrame(
    [
        {
            "country": feature["properties"]["ADMIN"],
        }
        for feature in countries_geojson["features"]
    ]
)
df_countries_geojson["country"].sort_values()

1                       Afghanistan
249    Akrotiri Sovereign Base Area
5                             Aland
4                           Albania
66                          Algeria
                   ...             
248               Wallis and Futuna
193                  Western Sahara
251                           Yemen
253                          Zambia
254                        Zimbabwe
Name: country, Length: 255, dtype: object

In [7]:
print(*df_countries_geojson["country"].sort_values().values, sep="\n")

Afghanistan
Akrotiri Sovereign Base Area
Aland
Albania
Algeria
American Samoa
Andorra
Angola
Anguilla
Antarctica
Antigua and Barbuda
Argentina
Armenia
Aruba
Ashmore and Cartier Islands
Australia
Austria
Azerbaijan
Bahrain
Bajo Nuevo Bank (Petrel Is.)
Bangladesh
Barbados
Baykonur Cosmodrome
Belarus
Belgium
Belize
Benin
Bermuda
Bhutan
Bolivia
Bosnia and Herzegovina
Botswana
Brazil
British Indian Ocean Territory
British Virgin Islands
Brunei
Bulgaria
Burkina Faso
Burundi
Cambodia
Cameroon
Canada
Cape Verde
Cayman Islands
Central African Republic
Chad
Chile
China
Clipperton Island
Colombia
Comoros
Cook Islands
Coral Sea Islands
Costa Rica
Croatia
Cuba
Curaçao
Cyprus
Cyprus No Mans Area
Czech Republic
Democratic Republic of the Congo
Denmark
Dhekelia Sovereign Base Area
Djibouti
Dominica
Dominican Republic
East Timor
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Ethiopia
Falkland Islands
Faroe Islands
Federated States of Micronesia
Fiji
Finland
France
French Polynesia
French S